## 事前準備

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/nlp100_2025/ja/

import os

current_directory = os.getcwd()
print(f"The Current Directory: {current_directory}")

# 第3章: 正規表現
Wikipediaの記事を以下のフォーマットで書き出したファイル[jawiki-country.json.gz](/data/jawiki-country.json.gz)がある。

* 1行に1記事の情報がJSON形式で格納される
* 各行には記事名が"title"キーに、記事本文が"text"キーの辞書オブジェクトに格納され、そのオブジェクトがJSON形式で書き出される
* ファイル全体はgzipで圧縮される

以下の処理を行うプログラムを作成せよ。

## 20. JSONデータの読み込み
Wikipedia記事のJSONファイルを読み込み、「イギリス」に関する記事本文を表示せよ。問題21-29では、ここで抽出した記事本文に対して実行せよ。

In [ ]:
import json
text_list = []
with open('../data/jawiki-country.json') as f:
    lines = f.readlines()
    for line in lines:
        text_list.append(json.loads(line))

In [ ]:
for i in range(len(text_list)):
    if text_list[i]['title'] == 'イギリス':
        # UK_text = str(text_list[i])
        UK_json = text_list[i]
        break

In [ ]:
UK_text = str(UK_json['text'])
UK_text[:200]

## 21. カテゴリ名を含む行を抽出
記事中でカテゴリ名を宣言している行を抽出せよ。

In [ ]:
import re
pattern = re.compile(r'\[\[Category:.*?\]\]')
result = pattern.findall(UK_text)
result

## 22. カテゴリ名の抽出
記事のカテゴリ名を（行単位ではなく名前で）抽出せよ。

In [ ]:
for item in result:
    category_str = item.replace('[[Category:', '').replace(']]', '').replace('|*', '')
    print(category_str)

## 23. セクション構造
記事中に含まれるセクション名とそのレベル（例えば"== セクション名 =="なら1）を表示せよ。

In [ ]:
import re
pattern = re.compile(r'==.*==')
result = pattern.findall(UK_text)
result
for item in result:
    level = int(len(re.findall(r'=', item))/2-1)
    print(f'{item} -> {level}')

## 24. ファイル参照の抽出
記事から参照されているメディアファイルをすべて抜き出せ。

In [ ]:
import re
pattern = re.compile(r'ファイル:(.+?)\|')
result = pattern.findall(UK_text)
print('\n'.join(result))

## 25. テンプレートの抽出
記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し、辞書オブジェクトとして格納せよ。

In [ ]:
import re
pattern = re.compile(r'\|(.+?)\s=\s*(.+)')
result = pattern.findall(UK_text)

In [ ]:
fundamental_info = {}
for item in result:
    print(item[0],item[1])
    fundamental_info[item[0]] = item[1]
fundamental_info

In [ ]:
fundamental_info['GDP/人']

## 26. 強調マークアップの除去
25の処理時に、テンプレートの値からMediaWikiの強調マークアップ（弱い強調、強調、強い強調のすべて）を除去してテキストに変換せよ（参考: [マークアップ早見表](http://ja.wikipedia.org/wiki/Help:%E6%97%A9%E8%A6%8B%E8%A1%A8)）。

In [ ]:
fundamental_info = {}
for item in result:
    # print(item[0],item[1])
    fundamental_info[item[0]] = item[1]
    match_sub = re.sub("\'{2,}(.+?)\'{2,}", "\\1", ''.join(item))
    print(match_sub)

## 27. 内部リンクの除去
26の処理に加えて、テンプレートの値からMediaWikiの内部リンクマークアップを除去し、テキストに変換せよ（参考: [マークアップ早見表](http://ja.wikipedia.org/wiki/Help:%E6%97%A9%E8%A6%8B%E8%A1%A8)）。

In [ ]:
fundamental_info = {}
for item in result:
    # print(item[0],item[1])
    fundamental_info[item[0]] = item[1]
    match_sub = re.sub(r"\'{2,}(.+?)\'{2,}", "\\1", ''.join(item))
    match_sub = re.sub(r"\[\[(.+?)\]\]", "\\1", match_sub)
    print(match_sub)

## 28. MediaWikiマークアップの除去
27の処理に加えて、テンプレートの値からMediaWikiマークアップを可能な限り除去し、国の基本情報を整形せよ。

In [ ]:
fundamental_info = {}
for item in result:
    # print(item[0],item[1])
    fundamental_info[item[0]] = item[1]
    match_sub = re.sub(r"\'{2,}(.+?)\'{2,}", "\\1", ''.join(item))
    match_sub = re.sub(r"\[\[(.+?)\]\]", "\\1", match_sub) #内部リンクマークアップの削除
    match_sub = re.sub(r"\[(.+?)\]", "\\1", match_sub) #外部リンクマークアップの削除
    match_sub = re.sub(r"\*+(.+?)", "\\1", match_sub) #*箇条書きの削除
    match_sub = re.sub(r"\:+(.+?)", "\\1", match_sub)#定義の箇条書きの削除
    match_sub = re.sub(r"\{\{(.+?)\}\}", "\\1", match_sub) #スタブなどを削除
    print(match_sub)

## 29. 国旗画像のURLを取得する
テンプレートの内容を利用し、国旗画像のURLを取得せよ。（ヒント: [MediaWiki API](http://www.mediawiki.org/wiki/API:Main_page/ja)の[imageinfo](https://www.mediawiki.org/wiki/API:Imageinfo)を呼び出して、ファイル参照をURLに変換すればよい）

In [ ]:
import requests

S = requests.Session()
URL = "https://en.wikipedia.org/w/api.php"

PARAMS = {
    "action": "query",
    "format": "json",
    "prop": "imageinfo",
    "titles": f"File:{fundamental_info['国旗画像']}",
    "iiprop":"url"
}

R = S.get(url=URL, params=PARAMS, verify=False) #ZCCが邪魔するので、False指定が必要

DATA = R.json()
PAGES = DATA["query"]["pages"]

for k, v in PAGES.items():
    print(v["imageinfo"][0]["url"])

In [ ]:
import requests
from IPython.display import SVG, Image # Import both just in case, but we'll use SVG for this link

image_url = "https://upload.wikimedia.org/wikipedia/en/a/ae/Flag_of_the_United_Kingdom.svg"

response = requests.get(image_url, verify=False)
display(SVG(data=response.content))